In [ ]:
from numpy.core.numerictypes import ScalarType
from google.colab.patches import cv2_imshow
import numpy as np
import cv2
import os
import glob

def leaf_area_calculation(img, cx, cy, w, h):
    # 센터 좌표에서 왼쪽 상단 꼭지점 좌표로 변환
    xtl = int(cx - w / 2)
    ytl = int(cy - h / 2)
    # 커널 생성-Image sharpening
    kernel = np.array([[-1, -1, -1],
                       [-1, 9, -1],
                       [-1, -1, -1]])

    # 커널 적용
    img_sharp = cv2.filter2D(img, -1, kernel)

    # 초기 마스크 생성
    mask = np.zeros(img_sharp.shape[:2],np.uint8)

    # 그랩컷에 사용할 임시 배열 생성
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)

    rect = (xtl,ytl,w,h) # 사각형 좌표 (추후에 Bounding Box 좌표를 대입)

    cv2.grabCut(img_sharp,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)

    # 배경인 곳은 0, 그 외에는 1로 설정한 마스크 생성
    mask2 = np.where((mask==2) | (mask==0), 0, 1).astype('uint8')

    # 이미지에 새로운 마스크를 곱해 배경을 제외
    img2 = img_sharp * mask2[:, :, np.newaxis]

    # 바이너리 이미지로 변환
    imgray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    ret, imthres = cv2.threshold(imgray, 10, 255, cv2.THRESH_BINARY)

    # 컨투어 찾기
    contour, hierarchy = cv2.findContours(imthres, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    area_max = 0
    # 각 도형의 컨투어에 대한 루프
    for c in contour:
        cv2.drawContours(img2, contour, -1, (0, 255, 0), 4)

        # 함수로 컨투어 넓이 계산해서 출력
        area = cv2.contourArea(c, False)
        if area_max < area:
            area_max = area
    
    print('leaf area:{:0.2f}'.format(area_max))
    # 결과 출력
    cv2_imshow(img2)
    return area_max

In [ ]:
from posixpath import split
disease_area = 0
leaf_area = 0
cx = cy = w = h = 0
i = 0
disease_percent = 0;

# 바운딩박스 좌표를 추출해 저장한 파일과 이미지를 하나씩 불러오기
# 이미지와 좌표 파일의 일치를 위해 정렬 후 불러오기
for file_img, file_label in zip(sorted(glob.glob('/content/drive/MyDrive/images_test/*.jpg')), sorted(glob.glob('/content/drive/MyDrive/labels_test/*.txt'))):
    img = cv2.imread(file_img)  # 이미지를 읽어와 img에 저장
    img_h, img_w, img_c = img.shape # 이미지 전체 크기 추출(좌표계 변환을 위함)
    with open(os.path.join(os.getcwd(), file_label), 'r') as f: # 바운딩박스 좌표 파일 읽어오기
        # 데이터를 스플릿해서 저장
        data = f.read().splitlines()
        split_Data = [x.split() for x in data]
        row = len(split_Data)
        col = len(split_Data[0])
        
        # 한 행에 저장된 정보는 [라벨, center X, center Y, width, height]
        # yolov5 좌표계에서 이미지 좌표계로 변환
        for i in range(row):
            cx = int(float(split_Data[i][1])*img_w)
            cy = int(float(split_Data[i][2])*img_h)
            w = int(float(split_Data[i][3])*img_w)
            h = int(float(split_Data[i][4])*img_h)
            if split_Data[i][0] == "1": # 라벨이 1일때(잎의 바운딩박스 좌표일때)
                leaf_area = leaf_area_calculation(img, cx, cy, w, h)     # 잎의 전체 면적 계산
            else:   # 라벨이 0일때(질병의 바운딩박스 좌표일때)
                disease_area += w * h   # 질병의 전체 면적 계산
    print('disease area:{:0.2f}'.format(disease_area))
    # 잎의 전체 면적이 구해진 경우 진행도 계산 후 출력
    if leaf_area != 0:
        disease_percent = disease_area / leaf_area * 100
        if disease_percent < 20:
            print("초기")
        elif 20 <= disease_percent < 50:
            print("중기")
        else:
            print("말기")
        print("\n\n")
    # 잎을 찾아내지 못했을 경우
    else:
        print("Can't Found Leaf")
    leaf_area = disease_area = 0    # 잎과 질병 면적 초기화

In [ ]:
from google.colab import drive
drive.mount('/content/drive')